In [2]:
import pandas as pd
import os
import numpy as np

In [21]:
# Newly Registered business data
df = pd.read_csv("/Users/nigel/SFPropertyTax/Registered_Business_Locations_-_San_Francisco.csv")

# Concats all years in tax data directory to calculate neighborhood appreciation
sf_tax_path = "/Users/nigel/SFPropertyTax/data/"
tax_df = pd.concat([pd.read_csv(sf_tax_path + df) for df in os.listdir(sf_tax_path)], sort=True)
tax_df["RP1LNDVAL"] = tax_df["RP1LNDVAL"].astype(float)

# Only select taxed values > $0
tax_df = tax_df[tax_df["RP1LNDVAL"] > 0][['RP1LNDVAL', 'MAZIPCODE', "ROLLYEAR"]]

# Convert decimal year to string
tax_df['ROLLYEAR'] = (2000 + tax_df['ROLLYEAR']).astype(str).str[:-2].astype(float)



# median_pricing = median_pricing[median_pricing.index.get_level_values(1) != "2007"]

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (9,32) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,MAZIPCODE,ROLLYEAR,RP1LNDVAL
0,BT7,2007.0,538342.0
1,BT7,2008.0,549108.0
2,BT7,2009.0,560090.0
3,BT7,2010.0,558763.0
4,BT7,2011.0,562968.0
5,CAN,2007.0,123704.0
6,CAN,2008.0,126178.0
7,CAN,2009.0,128701.0
8,T2T,2008.0,962370.0
9,T2T,2009.0,981617.0


In [52]:
# Group by zip code and year to get median price
median_pricing = tax_df.groupby(["MAZIPCODE", "ROLLYEAR"]).median()
median_pricing = median_pricing.reset_index()

# Calculate percent in appreciation/depreciation 
median_pricing["pct_change"] = np.where((median_pricing['MAZIPCODE'] == median_pricing['MAZIPCODE'].shift(1)) &
                                        (median_pricing['ROLLYEAR'] == median_pricing['ROLLYEAR'].shift(1) + 1)
                                        ,median_pricing['RP1LNDVAL'] / median_pricing['RP1LNDVAL'].shift(1) - 1, np.nan)


land_value_counts = tax_df.groupby(["MAZIPCODE", "ROLLYEAR"]).count().reset_index()
median_pricing = median_pricing.merge(land_value_counts, on=["MAZIPCODE", "ROLLYEAR"])
median_pricing = median_pricing.rename(columns={"RP1LNDVAL_x": "RP1LNDVAL", "RP1LNDVAL_y":"land_value_counts"})
median_pricing['previous year land value count'] = np.where((median_pricing['ROLLYEAR'] == median_pricing['ROLLYEAR'].shift(1) + 1) &
                                                            (median_pricing["MAZIPCODE"] == median_pricing['MAZIPCODE'].shift(1)),
                                                            median_pricing['land_value_counts'].shift(1), np.nan)


# Eliminated median prices with insufficient number of tax value samples. Couldn't get a good median value with insufficient samples
median_pricing = median_pricing[(median_pricing['land_value_counts'] > 200) & (median_pricing['previous year land value count'] > 200)]
median_pricing.sort_values("pct_change").head()

,MAZIPCODE,ROLLYEAR,RP1LNDVAL,pct_change,land_value_counts,previous year land value count
22178,94104,2014.0,121132.0,-0.508594,1000,868.0
22177,94104,2013.0,246501.0,-0.476113,868,638.0
13682,80129,2014.0,26063.0,-0.411285,535,231.0
10469,60062,2009.0,1800.0,-0.408478,1180,1126.0
10758,60606,2013.0,17995.0,-0.303114,312,543.0


In [57]:
df['Business Start Date'] = pd.to_datetime(sf_business['Business Start Date'])
sf_business = df[(df['City'] == "San Francisco") & (df['Business Start Date'] > '2007-01-01')]

new_businesses_count = sf_business.groupby([sf_business['Mail Zipcode'], sf_business['Business Start Date'].dt.year])['Transient Occupancy Tax'].count()
new_businesses_count.head()

Mail Zipcode  Business Start Date
-             2016                   1
01026         2019                   1
01095         2018                   1
01474         2007                   1
01535         2017                   1
Name: Transient Occupancy Tax, dtype: int64

In [61]:
new_median_pricing = median_pricing.reset_index()
reset_businesses_count = pd.DataFrame({'new business count': sf_business.groupby([sf_business['Mail Zipcode'], sf_business['Business Start Date'].dt.year]).size()}).reset_index()
reset_businesses_count.head()
new_median_pricing['ROLLYEAR'] = new_median_pricing['ROLLYEAR'].astype(int)
reset_businesses_count['Business Start Date'] = reset_businesses_count['Business Start Date'].astype(int)

In [62]:
merged_df = pd.merge(new_median_pricing, reset_businesses_count, how='left', left_on=['ROLLYEAR','MAZIPCODE'], right_on=['Business Start Date','Mail Zipcode'])
merged_df.head()

,index,MAZIPCODE,ROLLYEAR,RP1LNDVAL,pct_change,land_value_counts,previous year land value count,Mail Zipcode,Business Start Date,new business count
0,5215,22314,2008,41505.0,0.019979,244,243.0,NaN,NaN,NaN
1,5216,22314,2009,44137.0,0.063414,247,244.0,NaN,NaN,NaN
2,10468,60062,2008,3043.0,0.019772,1126,1092.0,NaN,NaN,NaN
3,10469,60062,2009,1800.0,-0.408478,1180,1126.0,NaN,NaN,NaN
4,10470,60062,2010,1800.0,0.000000,1207,1180.0,NaN,NaN,NaN


In [63]:
final_df = merged_df[['ROLLYEAR', "Mail Zipcode", "new business count", "pct_change", "RP1LNDVAL"]].dropna()

In [67]:
final_df = final_df.rename(columns={"RP1LNDVAL":"median land value price"}).sort_values("new business count", ascending=False)
final_df.head()

,ROLLYEAR,Mail Zipcode,new business count,pct_change,median land value price
152,2014,94110,418.0,0.062268,242810.0
151,2013,94110,399.0,0.092849,228577.0
109,2013,94103,396.0,0.068476,302853.0
150,2012,94110,369.0,0.104932,209157.0
130,2013,94107,361.0,0.118523,313200.0


In [19]:
by_pct_change = final_df.groupby(final_df["pct_change"].apply(lambda x: round(x, 1)))['new business count'].describe().sort_index(ascending=False)
by_pct_change[by_pct_change['count'] > 30]

,count,mean,std,min,25%,50%,75%,max
pct_change,,,,,,,,
0.3,35.0,15.857143,48.926201,1.0,2.0,3.0,5.5,247.0
0.2,77.0,11.688312,33.514828,1.0,1.0,3.0,7.0,204.0
0.1,247.0,57.481781,88.170374,1.0,3.0,9.0,92.5,418.0
0.0,293.0,41.767918,74.393719,1.0,2.0,5.0,29.0,328.0
-0.1,55.0,14.381818,36.010539,1.0,2.0,3.0,7.5,207.0


In [14]:
median_pricing

,MAZIPCODE,ROLLYEAR,RP1LNDVAL,pct_change,land_value_counts
208,00000,2007.0,119474.0,NaN,75
209,00000,2008.0,124166.5,0.039276,66
210,00000,2009.0,124300.0,0.001075,61
211,00000,2010.0,126349.0,0.016484,56
212,00000,2011.0,133737.0,0.058473,51
213,00000,2012.0,136411.0,0.019994,51
2338,10019,2011.0,106203.0,-0.950513,169
2339,10019,2012.0,109691.0,0.032843,135
2435,10038,2008.0,1101122.5,-0.896148,58
2436,10038,2009.0,2324043.0,1.110613,59


In [15]:
m

NameError: name 'm' is not defined

In [16]:
land_value_counts = tax_df.groupby(["MAZIPCODE", "ROLLYEAR"]).count().reset_index()
land_value_counts.head()

,MAZIPCODE,ROLLYEAR,RP1LNDVAL
0,BT7,2007.0,1
1,BT7,2008.0,1
2,BT7,2009.0,1
3,BT7,2010.0,1
4,BT7,2011.0,1


In [17]:
median_pricing[median_pricing['land_value_counts'] > 20]

,MAZIPCODE,ROLLYEAR,RP1LNDVAL,pct_change,land_value_counts
208,00000,2007.0,119474.0,NaN,75
209,00000,2008.0,124166.5,0.039276,66
210,00000,2009.0,124300.0,0.001075,61
211,00000,2010.0,126349.0,0.016484,56
212,00000,2011.0,133737.0,0.058473,51
213,00000,2012.0,136411.0,0.019994,51
2338,10019,2011.0,106203.0,-0.950513,169
2339,10019,2012.0,109691.0,0.032843,135
2435,10038,2008.0,1101122.5,-0.896148,58
2436,10038,2009.0,2324043.0,1.110613,59
